In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt

### 19-20 주중/주말 pickle 파일 불러오기

In [3]:
with open("../dataset/2019_week.pkl","rb") as fw:
    df2019_week=pickle.load(fw)

with open("../dataset/2019_hol.pkl","rb") as fw:
    df2019_hol=pickle.load(fw)
    
with open("../dataset/2020_week.pkl","rb") as fw:
    df2020_week=pickle.load(fw)
    
with open("../dataset/2020_hol.pkl","rb") as fw:
    df2020_hol=pickle.load(fw)

In [24]:
df2019_week.head()

,rent_id,rent_time,return_id,return_time,travel_time,travel_distance,fare,sex,age,type,times,year,month,day,dayofweek,hour
0,1421,2019-01-01 00:00:44,1420,2019-01-01 00:03:53,189.0,580.0,BIL_004,\N,AGE_002,Regular,1H,2019,1,1,Tue,0
1,1408,2019-01-01 00:02:16,1433,2019-01-01 00:07:07,291.0,1020.0,BIL_002,F,AGE_005,Regular,1H,2019,1,1,Tue,0
2,347,2019-01-01 00:04:12,387,2019-01-01 00:07:07,175.0,550.0,BIL_014,M,AGE_003,Regular,2H,2019,1,1,Tue,0
3,615,2019-01-01 00:03:09,612,2019-01-01 00:07:27,258.0,700.0,BIL_002,\N,AGE_002,Regular,1H,2019,1,1,Tue,0
4,1177,2019-01-01 00:01:44,1152,2019-01-01 00:08:09,385.0,1090.0,BIL_006,\N,AGE_002,Daily,1H,2019,1,1,Tue,0


In [38]:
df2019_week.groupby('rent_id')['type'].value_counts(normalize=True).unstack()

type,Daily,Regular
rent_id,,
1001,0.178073,0.821927
1002,0.190514,0.809486
1003,0.158406,0.841594
1004,0.176554,0.823446
1006,0.115788,0.884212
...,...,...
9993,0.059322,0.940678
9997,0.625000,0.375000
9998,0.357143,0.642857


In [6]:
# 대여소 기준 연도에 따른 주중/주말 이용권 비율 데이터프레임 생성

def ratio_by_station(df):
    grouped=df.groupby('rent_id')['type'].value_counts(normalize=True).unstack()
    
    station_id=grouped.index
    daily=grouped['Daily'].values
    regular=grouped['Regular'].values
    
    new_df=pd.DataFrame()
    new_df['station_id']=station_id
    new_df['Daily']=daily
    new_df['Regular']=regular
    
    return new_df

In [33]:
df2019_week_station=ratio_by_station(df2019_week)
df2019_week_station.rename(columns={'Daily':'19_week_daily', 'Regular':'19_week_regular'}, inplace=True)
df2019_hol_station=ratio_by_station(df2019_hol)
df2019_hol_station.rename(columns={'Daily':'19_hol_daily', 'Regular':'19_hol_regular'}, inplace=True)
df2020_week_station=ratio_by_station(df2020_week)
df2020_week_station.rename(columns={'Daily':'20_week_daily', 'Regular':'20_week_regular'}, inplace=True)
df2020_hol_station=ratio_by_station(df2020_hol)
df2020_hol_station.rename(columns={'Daily':'20_hol_daily', 'Regular':'20_hol_regular'}, inplace=True)

df_station=pd.merge(df2019_week_station, df2019_hol_station, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_week_station, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_hol_station, left_on='station_id',right_on='station_id',how='outer')

In [54]:
print('19년도 주중 : ',df2019_week_station.shape)
print('19년도 주말 : ',df2019_hol_station.shape)
print('20년도 주중 : ',df2020_week_station.shape)
print('20년도 주말 : ',df2020_hol_station.shape)

19년도 주중 :  (1559, 3)
19년도 주말 :  (1556, 3)
20년도 주중 :  (2164, 3)
20년도 주말 :  (2105, 3)


In [55]:
df_station=df_station.reset_index(drop=True)
df_station.head(20)

,station_id,19_week_daily,19_week_regular,19_hol_daily,19_hol_regular,20_week_daily,20_week_regular,20_hol_daily,20_hol_regular
0,1001,0.172905,0.827095,0.316646,0.683354,0.258624,0.741376,0.392308,0.607692
1,1002,0.183894,0.816106,0.356350,0.643650,0.283254,0.716746,0.454603,0.545397
2,1003,0.154073,0.845927,0.259140,0.740860,0.238314,0.761686,0.364192,0.635808
3,1004,0.172014,0.827986,0.301563,0.698438,0.272103,0.727897,0.428854,0.571146
4,1006,0.113201,0.886799,0.241314,0.758686,0.197472,0.802528,0.357414,0.642586
5,1007,0.212272,0.787728,0.443936,0.556064,0.272926,0.727074,0.541985,0.458015
6,1008,0.159479,0.840521,0.293281,0.706719,0.242745,0.757255,0.403878,0.596122
7,1009,0.204528,0.795472,0.341662,0.658338,0.266499,0.733501,0.417925,0.582075
8,101,0.147218,0.852782,0.248787,0.751213,0.191620,0.808380,0.306781,0.693219
9,1010,0.142545,0.857455,0.230505,0.769495,0.198139,0.801861,0.319519,0.680481


In [56]:
print('Outer join 이후, 대여소 목록 : ',df_station.shape)

Outer join 이후, 대여소 목록 :  (1527, 9)


In [57]:
sta_info=pd.read_csv('../dataset/con_sta_info.csv')
sta_info.head()

,ID,stationName,Y,X,Xn,Yn,Elev,BikeRoadNM,ToBikeRoad
0,101,101. (구)합정동 주민센터,37.549561,126.905754,947509.050934,1.950191e+06,9.152323,한강자전거길,771.0
1,102,102. 망원역 1번출구 앞,37.555649,126.910629,947943.924406,1.950863e+06,6.963737,홍제천자전거길,1065.0
2,103,103. 망원역 2번출구 앞,37.554951,126.910835,947961.634042,1.950786e+06,6.865565,한강자전거길,1146.0
3,104,104. 합정역 1번출구 앞,37.550629,126.914986,948325.239132,1.950304e+06,18.136753,한강자전거길,676.0
4,105,105. 합정역 5번출구 앞,37.550007,126.914825,948310.657239,1.950235e+06,17.498430,한강자전거길,962.0


In [58]:
print('대여소 정보 dataset : ',sta_info.shape)

대여소 정보 dataset :  (1527, 9)


In [59]:
id_info=sta_info['ID'].unique().tolist()
id_info=[str(i) for i in id_info]
id_info[:5]

['101', '102', '103', '104', '105']

In [60]:
type(id_info[0])

str

In [61]:
df_station=df_station[df_station['station_id'].isin(id_info)].reset_index(drop=True)
df_station.head()

,station_id,19_week_daily,19_week_regular,19_hol_daily,19_hol_regular,20_week_daily,20_week_regular,20_hol_daily,20_hol_regular
0,1001,0.172905,0.827095,0.316646,0.683354,0.258624,0.741376,0.392308,0.607692
1,1002,0.183894,0.816106,0.356350,0.643650,0.283254,0.716746,0.454603,0.545397
2,1003,0.154073,0.845927,0.259140,0.740860,0.238314,0.761686,0.364192,0.635808
3,1004,0.172014,0.827986,0.301563,0.698438,0.272103,0.727897,0.428854,0.571146
4,1006,0.113201,0.886799,0.241314,0.758686,0.197472,0.802528,0.357414,0.642586


In [62]:
print('이용권 비율을 확인 할 대여소 목록 :',df_station.shape)

이용권 비율을 확인 할 대여소 목록 : (1527, 9)


## 19->20 차이가 큰 정류소 찾기

#### 편하게 보기위해서 정기권만 가지고 확인

In [63]:
df_station_reg=df_station[['station_id','19_week_regular','19_hol_regular','20_week_regular','20_hol_regular']]
df_station_reg.head()

,station_id,19_week_regular,19_hol_regular,20_week_regular,20_hol_regular
0,1001,0.827095,0.683354,0.741376,0.607692
1,1002,0.816106,0.643650,0.716746,0.545397
2,1003,0.845927,0.740860,0.761686,0.635808
3,1004,0.827986,0.698438,0.727897,0.571146
4,1006,0.886799,0.758686,0.802528,0.642586


In [64]:
df_station_reg.shape

(1527, 5)

In [65]:
df_station_reg['week diff']=abs(df_station_reg['20_week_regular']-df_station_reg['19_week_regular'])
df_station_reg['hol diff']=abs(df_station_reg['20_hol_regular']-df_station_reg['19_hol_regular'])

df_station_reg.head()

C:\Users\river\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\river\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,station_id,19_week_regular,19_hol_regular,20_week_regular,20_hol_regular,week diff,hol diff
0,1001,0.827095,0.683354,0.741376,0.607692,0.085718,0.075662
1,1002,0.816106,0.643650,0.716746,0.545397,0.099361,0.098253
2,1003,0.845927,0.740860,0.761686,0.635808,0.084241,0.105052
3,1004,0.827986,0.698438,0.727897,0.571146,0.100089,0.127291
4,1006,0.886799,0.758686,0.802528,0.642586,0.084271,0.116101


In [97]:
def make_sign(data1, data2):
    if data1 > data2 :
        return '-'
    elif data2 > data1 :
        return '+'
    else:
        return '0'

In [67]:
df_station_reg['week diff sign']=df_station_reg.apply(lambda x:make_sign(x['19_week_regular'], x['20_week_regular']), axis=1)
df_station_reg['hol diff sign']=df_station_reg.apply(lambda x:make_sign(x['19_hol_regular'], x['20_hol_regular']), axis=1)

df_station_reg.head()

C:\Users\river\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,station_id,19_week_regular,19_hol_regular,20_week_regular,20_hol_regular,week diff,hol diff,week diff sign,hol diff sign
0,1001,0.827095,0.683354,0.741376,0.607692,0.085718,0.075662,-,-
1,1002,0.816106,0.643650,0.716746,0.545397,0.099361,0.098253,-,-
2,1003,0.845927,0.740860,0.761686,0.635808,0.084241,0.105052,-,-
3,1004,0.827986,0.698438,0.727897,0.571146,0.100089,0.127291,-,-
4,1006,0.886799,0.758686,0.802528,0.642586,0.084271,0.116101,-,-


In [68]:
df_station_reg['week diff sign'].value_counts()

-    1496
+      31
Name: week diff sign, dtype: int64

In [69]:
# 정기권의 이용 비율이 증가한 대여소 (주중)

station_week_plus=df_station_reg[df_station_reg['week diff sign']=='+']['station_id'].tolist()
print(station_week_plus)
print('크기 : ',len(station_week_plus))

['1025', '1101', '1241', '1296', '1304', '1370', '1376', '1424', '1652', '1679', '1910', '198', '2141', '2180', '2225', '2275', '2357', '2364', '2385', '2392', '315', '322', '330', '337', '346', '3527', '447', '449', '462', '575', '802']
크기 :  31


In [70]:
13/1527

0.008513425016371971

In [71]:
df_station_reg['hol diff sign'].value_counts()

-    1472
+      55
Name: hol diff sign, dtype: int64

In [72]:
# 정기권의 이용 비율이 증가한 대여소 (주말)

station_hol_plus=df_station_reg[df_station_reg['hol diff sign']=='+']['station_id'].tolist()
print(station_hol_plus)
print('크기 : ',len(station_hol_plus))

['1108', '1220', '127', '1296', '1304', '1316', '136', '1370', '1457', '1510', '1670', '1910', '199', '1993', '2061', '2127', '2130', '2141', '2180', '2233', '2239', '2266', '2272', '2280', '2328', '2333', '2406', '2409', '2502', '2609', '2610', '2612', '307', '315', '322', '330', '336', '346', '3529', '3532', '354', '380', '391', '453', '454', '455', '462', '575', '580', '608', '640', '802', '812', '846', '963']
크기 :  55


In [75]:
# 정기권의 이용 비율이 증가한 대여소 교집합 (주중-주말)

plus_intersection=set(station_week_plus).intersection(set(station_hol_plus))
print(plus_intersection)
print('크기 : ',len(plus_intersection))

{'1296', '2180', '322', '315', '330', '2141', '802', '1910', '1370', '1304', '575', '462', '346'}
크기 :  13


In [24]:
df_station_reg['week diff'].describe()

count    1527.000000
mean        0.059412
std         0.030902
min         0.000593
25%         0.038381
50%         0.055540
75%         0.077492
max         0.207296
Name: week diff, dtype: float64

In [25]:
df_station_reg['hol diff'].describe()

count    1527.000000
mean        0.072593
std         0.039381
min         0.000163
25%         0.045215
50%         0.070242
75%         0.095023
max         0.284067
Name: hol diff, dtype: float64

### Quantile 별 소속 대여소 구분

#### 주중

In [164]:
week_qt1=df_station_reg['week diff'].quantile(.25)
week_qt2=df_station_reg['week diff'].quantile(.5)
week_qt3=df_station_reg['week diff'].quantile(.75)

In [165]:
station_week_qt1=df_station_reg[df_station_reg['week diff'] < week_qt1]['station_id'].tolist()
station_week_qt2=df_station_reg[(df_station_reg['week diff'] >= week_qt1)&(df_station_reg['week diff'] < week_qt2)]['station_id'].tolist()
station_week_qt3=df_station_reg[(df_station_reg['week diff'] >= week_qt2)&(df_station_reg['week diff'] < week_qt3)]['station_id'].tolist()
station_week_qt4=df_station_reg[df_station_reg['week diff'] >= week_qt3]['station_id'].tolist()

In [166]:
iqr=week_qt3-week_qt1
out_up=week_qt3+1.5*iqr
out_down=week_qt1-1.5*iqr

print(out_up, out_down)

0.1361574894727374 -0.02028431900019459


In [167]:
# 이상치로 간주된 정류소

station_outlier_week=df_station_reg[df_station_reg['week diff'] > out_up]['station_id'].tolist()
print(station_outlier_week)
print('크기 : ',len(station_outlier_week))

['1039', '1052', '1056', '1058', '1359', '1445', '1450', '1452', '1664', '1816', '1824', '1825', '1826', '1847', '1857', '1858', '1937', '1951', '1952', '1987', '2038', '2130', '221', '3525', '502', '524', '928', '938', '966']
크기 :  29


#### 주말

In [168]:
hol_qt1=df_station_reg['hol diff'].quantile(.25)
hol_qt2=df_station_reg['hol diff'].quantile(.5)
hol_qt3=df_station_reg['hol diff'].quantile(.75)

In [169]:
station_hol_qt1=df_station_reg[df_station_reg['hol diff'] < hol_qt1]['station_id'].tolist()
station_hol_qt2=df_station_reg[(df_station_reg['hol diff'] >= hol_qt1)&(df_station_reg['hol diff'] < hol_qt2)]['station_id'].tolist()
station_hol_qt3=df_station_reg[(df_station_reg['hol diff'] >= hol_qt2)&(df_station_reg['hol diff'] < hol_qt3)]['station_id'].tolist()
station_hol_qt4=df_station_reg[df_station_reg['hol diff'] >= hol_qt3]['station_id'].tolist()

In [170]:
iqr=hol_qt3-hol_qt1
out_up=hol_qt3+1.5*iqr
out_down=hol_qt1-1.5*iqr

print(out_up, out_down)

0.16973560896893147 -0.029498094943539402


In [171]:
# 이상치로 간주된 정류소

station_outlier_hol=df_station_reg[df_station_reg['hol diff'] > out_up]['station_id'].tolist()
print(station_outlier_hol)
print('크기 : ',len(station_outlier_hol))

['1038', '1057', '1058', '1406', '1421', '1819', '1826', '1835', '1836', '1847', '1857', '1858', '1929', '1952', '1953', '1969', '1999', '206', '2201', '2287', '2309', '2341', '3524', '3528', '3539', '827', '902']
크기 :  27


In [173]:
# 주중/주말 모두 변화가 큰 대여소

set(station_outlier_week).intersection(set(station_outlier_hol))

{'1058', '1826', '1847', '1857', '1858', '1952'}

### 10% 이상 이용비율의 변화가 있는 대여소 목록

#### 주중

In [26]:
station_over10_week=df_station_reg[df_station_reg['week diff'] >= 0.1]['station_id'].tolist()
print(station_over10_week)
print('크기 : ',len(station_over10_week))

['1004', '1037', '1038', '1039', '1052', '1053', '1056', '1057', '1058', '1122', '1161', '117', '119', '123', '1271', '1275', '1291', '131', '1315', '1332', '1347', '1357', '1359', '1373', '1413', '1420', '1434', '1437', '1438', '1445', '1450', '1452', '1453', '1512', '152', '1533', '1534', '1548', '1620', '1640', '1641', '1659', '1661', '1664', '1667', '1677', '1691', '1715', '1717', '1720', '1726', '1730', '1735', '1736', '1738', '1814', '1816', '182', '1820', '1821', '1824', '1825', '1826', '1827', '1836', '1846', '1847', '1855', '1857', '1858', '186', '1925', '1928', '1929', '1937', '1951', '1952', '1981', '1987', '1996', '2024', '2038', '205', '2054', '207', '209', '2103', '2107', '2130', '215', '2159', '221', '2214', '2215', '2217', '2220', '2262', '2265', '227', '2308', '2309', '2344', '2351', '2356', '2373', '2389', '2403', '2411', '248', '249', '2603', '2605', '2616', '272', '3525', '418', '502', '510', '511', '523', '524', '534', '544', '556', '564', '565', '581', '606', '607

#### 주말

In [27]:
station_over10_hol=df_station_reg[df_station_reg['hol diff'] >= 0.1]['station_id'].tolist()
print(station_over10_hol)
print('크기 : ',len(station_over10_hol))

['1003', '1004', '1006', '1008', '1016', '1023', '1031', '1032', '1033', '1037', '1038', '1039', '1041', '1042', '1052', '1054', '1056', '1057', '1058', '1062', '1105', '111', '1120', '1152', '1155', '1160', '1178', '1188', '119', '1207', '121', '1226', '1249', '1250', '1263', '1271', '1280', '1291', '131', '1314', '1318', '1319', '1323', '1325', '1334', '1347', '1366', '1377', '1406', '1413', '1416', '1418', '1420', '1421', '1422', '1426', '1428', '1429', '1434', '1438', '1450', '1452', '148', '1511', '1512', '1513', '1514', '1528', '1533', '1539', '1605', '1606', '1610', '1620', '1634', '1639', '1640', '1646', '165', '1654', '1659', '166', '1661', '1664', '1668', '1671', '1672', '1674', '1678', '1679', '1680', '1685', '1703', '1714', '1715', '1717', '1722', '1723', '1726', '1727', '1728', '1729', '1730', '1734', '1736', '1738', '1742', '175', '1804', '1805', '1806', '1809', '1814', '1819', '1820', '1822', '1823', '1824', '1825', '1826', '1827', '1828', '1830', '1831', '1833', '1834',

In [29]:
# 교집합

station_over10_intersection=set(station_over10_week).intersection(set(station_over10_hol))
print(station_over10_intersection)
print('크기 : ',len(station_over10_intersection))

{'1037', '902', '2351', '606', '272', '1730', '1664', '1438', '1925', '1937', '1413', '1846', '1661', '1056', '2215', '1039', '2344', '607', '1824', '1058', '1533', '938', '1814', '1858', '1640', '209', '249', '221', '2103', '711', '2616', '1827', '227', '1836', '2107', '119', '712', '1004', '1726', '1929', '205', '1420', '2038', '1855', '2265', '524', '1825', '1736', '1987', '1434', '915', '2024', '928', '581', '1826', '2356', '1952', '2603', '544', '2389', '1450', '1738', '923', '1857', '714', '1347', '1452', '1659', '1291', '1052', '1847', '1715', '2214', '937', '534', '1981', '131', '2309', '1620', '1951', '663', '1271', '1820', '1512', '1038', '1057', '1717', '647', '966', '907'}
크기 :  90


## 반기에 따른 대여소 이용권 비율변화 확인

#### 반기

In [77]:
def make_half(df):
    first=[1,2,3,4,5,6]
    second=[7,8,9,10,11,12]
    
    df_first=df[df['month'].isin(first)].reset_index(drop=True)
    df_second=df[df['month'].isin(second)].reset_index(drop=True)
    
    return df_first, df_second

,rent_id,rent_time,return_id,return_time,travel_time,travel_distance,fare,sex,age,type,times,year,month,day,dayofweek,hour,month-day
0,437,2019-01-02 00:02:29,126,2019-01-02 00:04:44,135.0,410.0,BIL_004,M,AGE_003,Regular,1H,2019,1,2,Wed,0,0 1\n1 1\n2 1...
1,747,2019-01-02 00:00:41,726,2019-01-02 00:05:15,274.0,660.0,BIL_006,\N,AGE_002,Daily,1H,2019,1,2,Wed,0,0 1\n1 1\n2 1...
2,1149,2019-01-02 00:01:25,1112,2019-01-02 00:05:41,256.0,800.0,BIL_006,\N,AGE_002,Daily,1H,2019,1,2,Wed,0,0 1\n1 1\n2 1...
3,2025,2019-01-02 00:02:27,2040,2019-01-02 00:05:50,203.0,590.0,BIL_014,M,AGE_002,Regular,2H,2019,1,2,Wed,0,0 1\n1 1\n2 1...
4,1448,2019-01-02 00:01:08,1442,2019-01-02 00:05:57,289.0,1350.0,BIL_005,M,AGE_002,Regular,1H,2019,1,2,Wed,0,0 1\n1 1\n2 1...


In [78]:
df2019_week_one, df2019_week_two=make_half(df2019_week)
df2020_week_one, df2020_week_two=make_half(df2020_week)

In [82]:
print('19 주중 상반기 : ',df2019_week_one.shape)
print('19 주중 하반기 : ',df2019_week_two.shape)
print('20 주중 상반기 : ',df2020_week_one.shape)
print('20 주중 하반기 : ',df2020_week_two.shape)

19 주중 상반기 :  (5412075, 17)
19 주중 하반기 :  (7588630, 17)
20 주중 상반기 :  (3991300, 16)
20 주중 하반기 :  (1589180, 16)


In [84]:
df2019_hol_one, df2019_hol_two=make_half(df2019_hol)
df2020_hol_one, df2020_hol_two=make_half(df2020_hol)

In [85]:
print('19 주말 상반기 : ',df2019_hol_one.shape)
print('19 주말 하반기 : ',df2019_hol_two.shape)
print('20 주말 상반기 : ',df2020_hol_one.shape)
print('20 주말 하반기 : ',df2020_hol_two.shape)

19 주말 상반기 :  (2126506, 17)
19 주말 하반기 :  (2988974, 17)
20 주말 상반기 :  (1663405, 16)
20 주말 하반기 :  (702857, 16)


In [80]:
df2019_week_one.head()

,rent_id,rent_time,return_id,return_time,travel_time,travel_distance,fare,sex,age,type,times,year,month,day,dayofweek,hour,month-day
0,437,2019-01-02 00:02:29,126,2019-01-02 00:04:44,135.0,410.0,BIL_004,M,AGE_003,Regular,1H,2019,1,2,Wed,0,0 1\n1 1\n2 1...
1,747,2019-01-02 00:00:41,726,2019-01-02 00:05:15,274.0,660.0,BIL_006,\N,AGE_002,Daily,1H,2019,1,2,Wed,0,0 1\n1 1\n2 1...
2,1149,2019-01-02 00:01:25,1112,2019-01-02 00:05:41,256.0,800.0,BIL_006,\N,AGE_002,Daily,1H,2019,1,2,Wed,0,0 1\n1 1\n2 1...
3,2025,2019-01-02 00:02:27,2040,2019-01-02 00:05:50,203.0,590.0,BIL_014,M,AGE_002,Regular,2H,2019,1,2,Wed,0,0 1\n1 1\n2 1...
4,1448,2019-01-02 00:01:08,1442,2019-01-02 00:05:57,289.0,1350.0,BIL_005,M,AGE_002,Regular,1H,2019,1,2,Wed,0,0 1\n1 1\n2 1...


In [86]:
# 대여소 기준 연도에 따른 주중/주말 이용권 비율 데이터프레임 생성

def ratio_by_station(df):
    grouped=df.groupby('rent_id')['type'].value_counts(normalize=True).unstack()
    
    station_id=grouped.index
    #daily=grouped['Daily'].values
    regular=grouped['Regular'].values
    
    new_df=pd.DataFrame()
    new_df['station_id']=station_id
    #new_df['Daily']=daily
    new_df['Regular']=regular
    
    return new_df

In [89]:
df2019_week_station_one=ratio_by_station(df2019_week_one)
df2019_week_station_one.rename(columns={'Regular':'19_week_one'}, inplace=True)
df2019_week_station_two=ratio_by_station(df2019_week_two)
df2019_week_station_two.rename(columns={'Regular':'19_week_two'}, inplace=True)

df2020_week_station_one=ratio_by_station(df2020_week_one)
df2020_week_station_one.rename(columns={'Regular':'20_week_one'}, inplace=True)
df2020_week_station_two=ratio_by_station(df2020_week_two)
df2020_week_station_two.rename(columns={'Regular':'20_week_two'}, inplace=True)

df2019_hol_station_one=ratio_by_station(df2019_hol_one)
df2019_hol_station_one.rename(columns={'Regular':'19_hol_one'}, inplace=True)
df2019_hol_station_two=ratio_by_station(df2019_hol_two)
df2019_hol_station_two.rename(columns={'Regular':'19_hol_two'}, inplace=True)

df2020_hol_station_one=ratio_by_station(df2020_hol_one)
df2020_hol_station_one.rename(columns={'Regular':'20_hol_one'}, inplace=True)
df2020_hol_station_two=ratio_by_station(df2020_hol_two)
df2020_hol_station_two.rename(columns={'Regular':'20_hol_two'}, inplace=True)



df_station=pd.merge(df2019_week_station_one, df2019_week_station_two, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_week_station_one, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_week_station_two, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2019_hol_station_one, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2019_hol_station_two, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_hol_station_one, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_hol_station_two, left_on='station_id',right_on='station_id',how='outer')

In [90]:
df_station.head()

,station_id,19_week_one,19_week_two,20_week_one,20_week_two,19_hol_one,19_hol_two,20_hol_one,20_hol_two
0,1001,0.807945,0.839880,0.733678,0.760291,0.652327,0.704371,0.590000,0.649438
1,1002,0.782372,0.835476,0.704082,0.746063,0.587016,0.676923,0.538212,0.560711
2,1003,0.850483,0.842604,0.759312,0.768049,0.731156,0.748120,0.633615,0.641509
3,1004,0.805674,0.842854,0.718178,0.751351,0.693945,0.701326,0.558379,0.603873
4,1006,0.878187,0.892407,0.805315,0.791111,0.752126,0.762911,0.647128,0.623116


In [91]:
df_station.shape

(2183, 9)

In [92]:
df_station=df_station[df_station['station_id'].isin(id_info)].reset_index(drop=True)
df_station.head()

,station_id,19_week_one,19_week_two,20_week_one,20_week_two,19_hol_one,19_hol_two,20_hol_one,20_hol_two
0,1001,0.807945,0.839880,0.733678,0.760291,0.652327,0.704371,0.590000,0.649438
1,1002,0.782372,0.835476,0.704082,0.746063,0.587016,0.676923,0.538212,0.560711
2,1003,0.850483,0.842604,0.759312,0.768049,0.731156,0.748120,0.633615,0.641509
3,1004,0.805674,0.842854,0.718178,0.751351,0.693945,0.701326,0.558379,0.603873
4,1006,0.878187,0.892407,0.805315,0.791111,0.752126,0.762911,0.647128,0.623116


In [93]:
df_station.shape

(1527, 9)

In [95]:
df_station['week_one_diff']=abs(df_station['20_week_one']-df_station['19_week_one'])
df_station['week_two_diff']=abs(df_station['20_week_two']-df_station['19_week_two'])
df_station['hol_one_diff']=abs(df_station['20_hol_one']-df_station['19_hol_one'])
df_station['hol_two_diff']=abs(df_station['20_hol_two']-df_station['19_hol_two'])

df_station.head()

,station_id,19_week_one,19_week_two,20_week_one,20_week_two,19_hol_one,19_hol_two,20_hol_one,20_hol_two,week_one_diff,week_two_diff,hol_one_diff,hol_two_diff
0,1001,0.807945,0.839880,0.733678,0.760291,0.652327,0.704371,0.590000,0.649438,0.074266,0.079589,0.062327,0.054933
1,1002,0.782372,0.835476,0.704082,0.746063,0.587016,0.676923,0.538212,0.560711,0.078290,0.089413,0.048804,0.116212
2,1003,0.850483,0.842604,0.759312,0.768049,0.731156,0.748120,0.633615,0.641509,0.091171,0.074555,0.097540,0.106611
3,1004,0.805674,0.842854,0.718178,0.751351,0.693945,0.701326,0.558379,0.603873,0.087496,0.091503,0.135566,0.097453
4,1006,0.878187,0.892407,0.805315,0.791111,0.752126,0.762911,0.647128,0.623116,0.072872,0.101296,0.104999,0.139795


In [98]:
df_station['week one diff sign']=df_station.apply(lambda x:make_sign(x['19_week_one'], x['20_week_one']), axis=1)
df_station['week two diff sign']=df_station.apply(lambda x:make_sign(x['19_week_two'], x['20_week_two']), axis=1)
df_station['hol one diff sign']=df_station.apply(lambda x:make_sign(x['19_hol_one'], x['20_hol_one']), axis=1)
df_station['hol two diff sign']=df_station.apply(lambda x:make_sign(x['19_hol_two'], x['20_hol_two']), axis=1)

df_station.head()

,station_id,19_week_one,19_week_two,20_week_one,20_week_two,19_hol_one,19_hol_two,20_hol_one,20_hol_two,week_one_diff,week_two_diff,hol_one_diff,hol_two_diff,week one diff sign,week two diff sign,hol one diff sign,hol two diff sign
0,1001,0.807945,0.839880,0.733678,0.760291,0.652327,0.704371,0.590000,0.649438,0.074266,0.079589,0.062327,0.054933,-,-,-,-
1,1002,0.782372,0.835476,0.704082,0.746063,0.587016,0.676923,0.538212,0.560711,0.078290,0.089413,0.048804,0.116212,-,-,-,-
2,1003,0.850483,0.842604,0.759312,0.768049,0.731156,0.748120,0.633615,0.641509,0.091171,0.074555,0.097540,0.106611,-,-,-,-
3,1004,0.805674,0.842854,0.718178,0.751351,0.693945,0.701326,0.558379,0.603873,0.087496,0.091503,0.135566,0.097453,-,-,-,-
4,1006,0.878187,0.892407,0.805315,0.791111,0.752126,0.762911,0.647128,0.623116,0.072872,0.101296,0.104999,0.139795,-,-,-,-


In [100]:
# 정기권의 이용 비율이 증가한 대여소 (주중 상반기)

station_week_one_plus=df_station[df_station['week one diff sign']=='+']['station_id'].tolist()
print(station_week_one_plus)
print('크기 : ',len(station_week_one_plus))

['1007', '1025', '1101', '1177', '1186', '1187', '1194', '1195', '1196', '1211', '1220', '1241', '1293', '1296', '1297', '1304', '135', '1369', '1370', '1376', '1377', '140', '1501', '1513', '1536', '1546', '1646', '1652', '1662', '1679', '1693', '1731', '1806', '1910', '198', '1993', '2036', '2041', '2104', '2106', '2141', '2180', '2225', '2233', '2245', '2270', '2275', '2279', '2289', '2292', '2297', '2299', '2302', '2318', '2319', '2329', '2330', '2348', '2352', '2357', '2364', '2371', '2385', '2392', '2407', '2505', '2510', '2607', '2610', '2619', '315', '324', '330', '337', '346', '3504', '3527', '3529', '3536', '354', '379', '391', '411', '424', '428', '449', '462', '507', '508', '575', '639', '640', '656', '748', '802', '817', '846', '925', '963', '968', '972']
크기 :  101


In [101]:
# 정기권의 이용 비율이 증가한 대여소 (주중 하반기)

station_week_two_plus=df_station[df_station['week two diff sign']=='+']['station_id'].tolist()
print(station_week_two_plus)
print('크기 : ',len(station_week_two_plus))

['1025', '1054', '1239', '1241', '1267', '1268', '1297', '1304', '1376', '1510', '1511', '1513', '1547', '1652', '1680', '1714', '1728', '1943', '196', '1969', '2064', '218', '2226', '2242', '2333', '2343', '2391', '2392', '2609', '320', '331', '346', '3510', '394', '430', '455', '543', '575', '710', '831']
크기 :  40


In [103]:
# 정기권의 이용 비율이 증가한 대여소 (주말 상반기)

station_hol_one_plus=df_station[df_station['hol one diff sign']=='+']['station_id'].tolist()
print(station_hol_one_plus)
print('크기 : ',len(station_hol_one_plus))

['1027', '1061', '1108', '1109', '1114', '1117', '1135', '1168', '1182', '1186', '1190', '1191', '1194', '120', '1203', '1204', '1211', '1220', '1221', '1224', '1241', '1251', '1260', '1269', '127', '1283', '1293', '1296', '1297', '1302', '1304', '1316', '1321', '1326', '1333', '1338', '135', '1368', '137', '1370', '1376', '1377', '141', '1412', '1435', '1457', '1501', '1510', '1529', '1546', '1630', '1650', '1652', '1670', '1693', '1707', '1731', '181', '1843', '1910', '1913', '1946', '1967', '199', '1992', '1993', '200', '2036', '2058', '2061', '2068', '2104', '2127', '2141', '2180', '2210', '2231', '2233', '2239', '2244', '2255', '2258', '2266', '2271', '2272', '2277', '2280', '2292', '2297', '2299', '2302', '2303', '2315', '2319', '2323', '2328', '2333', '2335', '2357', '2360', '2362', '2369', '2371', '2380', '2385', '2390', '2392', '2395', '2406', '2409', '2502', '2503', '2601', '2606', '2608', '2610', '2612', '2619', '285', '307', '310', '315', '322', '330', '336', '346', '3502',

In [104]:
# 정기권의 이용 비율이 증가한 대여소 (주말 하반기)

station_hol_two_plus=df_station[df_station['hol two diff sign']=='+']['station_id'].tolist()
print(station_hol_two_plus)
print('크기 : ',len(station_hol_two_plus))

['1018', '1027', '1029', '1056', '1102', '1151', '1183', '1210', '1220', '1229', '1231', '1233', '1242', '1268', '1304', '1316', '136', '1362', '1365', '1370', '1444', '1457', '1510', '1534', '1545', '160', '1670', '1675', '171', '1731', '179', '1859', '196', '199', '2016', '2041', '2061', '2062', '2064', '2111', '2127', '2130', '2165', '2203', '2225', '2229', '2233', '2239', '2249', '2259', '2280', '2284', '2288', '2306', '2343', '2350', '2362', '2371', '2381', '2391', '2406', '242', '244', '2502', '2605', '2609', '267', '322', '331', '337', '3527', '3529', '354', '356', '368', '375', '380', '389', '394', '398', '406', '410', '447', '449', '454', '455', '462', '522', '543', '555', '564', '608', '610', '633', '634', '710', '746', '748', '802', '803', '806', '839', '845', '948']
크기 :  104


In [107]:
# 상반기 교집합

print(set(station_week_one_plus).intersection(set(station_hol_one_plus)))
print('크기 : ',len(set(station_week_one_plus).intersection(set(station_hol_one_plus))))

{'1546', '1297', '330', '1241', '2319', '1304', '2619', '925', '575', '1652', '1186', '2385', '1693', '2299', '135', '2180', '1293', '3529', '972', '315', '963', '1993', '1377', '354', '1194', '2233', '2302', '2104', '391', '1376', '2141', '2357', '1910', '1370', '2610', '1220', '1296', '1731', '846', '1501', '2392', '2292', '1211', '2297', '2036', '640', '2371', '346'}
크기 :  48


In [108]:
# 하반기 교집합

print(set(station_week_two_plus).intersection(set(station_hol_two_plus)))
print('크기 : ',len(set(station_week_two_plus).intersection(set(station_hol_two_plus))))

{'1268', '455', '710', '1510', '196', '2064', '2343', '1304', '2391', '2609', '394', '331', '543'}
크기 :  13


#### 10% 이상의 비율변화

In [109]:
station_over10_week_one=df_station[df_station['week_one_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_week_one)
print('크기 : ',len(station_over10_week_one))

['1016', '1037', '1039', '1052', '1053', '1054', '1056', '1057', '1058', '1122', '119', '1271', '1275', '1291', '1315', '1332', '1359', '1373', '1413', '1428', '1438', '1450', '1452', '1453', '1511', '152', '1545', '1548', '1602', '1620', '1628', '1641', '1654', '1659', '1661', '1664', '1667', '1672', '1677', '1715', '1717', '1720', '1723', '1726', '1730', '1735', '1816', '182', '1820', '1821', '1822', '1824', '1825', '1826', '1835', '1846', '1855', '1857', '1858', '1929', '1937', '1951', '1952', '1953', '1969', '1977', '1981', '1987', '199', '1996', '2024', '2038', '205', '2054', '207', '209', '2103', '2130', '2159', '2175', '2203', '2207', '221', '2214', '2229', '2262', '2265', '227', '2288', '2308', '2309', '2323', '2343', '2344', '2351', '2356', '2376', '2389', '2403', '248', '3525', '373', '418', '422', '429', '462', '502', '523', '524', '534', '536', '544', '556', '581', '606', '607', '711', '714', '803', '808', '825', '902', '922', '923', '928', '959', '966']
크기 :  127


In [111]:
station_over10_week_two=df_station[df_station['week_two_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_week_two)
print('크기 : ',len(station_over10_week_two))

['1006', '1007', '1038', '1049', '1053', '1057', '1058', '106', '1161', '117', '1170', '1181', '1188', '119', '123', '1271', '1318', '1325', '133', '1332', '1334', '1346', '1347', '1348', '1349', '1353', '1355', '1357', '1359', '1366', '1367', '1373', '1407', '1413', '1418', '1420', '1421', '1429', '1434', '1435', '1436', '1437', '1438', '1441', '1445', '1450', '1452', '1512', '152', '1528', '1534', '1537', '1541', '1548', '1640', '1641', '1659', '166', '1663', '1664', '1668', '1677', '1691', '170', '1702', '1708', '1715', '1720', '1722', '1726', '1734', '1736', '1738', '1741', '1742', '175', '178', '1805', '1814', '1819', '182', '1824', '1825', '1826', '1827', '1830', '1831', '1833', '1836', '1846', '1847', '1850', '1853', '1857', '1858', '1925', '1928', '1937', '194', '1951', '1952', '1965', '1967', '1987', '1988', '1991', '1992', '201', '2012', '2024', '2025', '2034', '2038', '2054', '2056', '2065', '207', '2102', '2103', '2107', '2128', '2130', '2135', '2140', '2159', '2169', '2172

In [112]:
station_over10_hol_one=df_station[df_station['hol_one_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_hol_one)
print('크기 : ',len(station_over10_hol_one))

['1004', '1006', '1008', '1015', '1016', '1017', '1023', '1031', '1032', '1033', '1038', '1039', '1041', '1052', '1053', '1054', '1056', '1057', '1058', '1105', '1120', '1161', '1172', '1178', '1183', '1188', '119', '1207', '121', '1226', '1228', '1250', '1263', '1266', '1271', '1280', '1285', '1291', '131', '1318', '1323', '1347', '1357', '1376', '1405', '1406', '1413', '1416', '1418', '1420', '1421', '1422', '1426', '1428', '1429', '1438', '1443', '1444', '1450', '148', '1511', '1528', '1617', '1620', '1639', '1640', '165', '1654', '1659', '1661', '1668', '1675', '1677', '1678', '1714', '1715', '1717', '1722', '1723', '1726', '1728', '1730', '1734', '1805', '1806', '1809', '1814', '1819', '1820', '1822', '1825', '1826', '1828', '1830', '1831', '1833', '1834', '1835', '1836', '1842', '1847', '185', '1851', '1852', '1855', '1857', '1858', '1859', '1907', '1916', '1924', '1929', '1930', '194', '1941', '1951', '1952', '1953', '1965', '1969', '1977', '1979', '1981', '1985', '1987', '1993'

In [116]:
df_station['hol_two_diff'].describe()

count    1519.000000
mean        0.079000
std         0.050338
min         0.000071
25%         0.043249
50%         0.073211
75%         0.107261
max         0.462097
Name: hol_two_diff, dtype: float64

In [113]:
station_over10_hol_two=df_station[df_station['hol_two_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_hol_two)
print('크기 : ',len(station_over10_hol_two))

['1002', '1003', '1006', '1007', '1010', '1016', '1031', '1032', '1033', '1037', '1038', '1042', '1049', '1051', '1052', '1057', '1058', '1061', '1062', '108', '1105', '111', '1115', '1124', '1137', '1145', '1148', '1152', '1155', '1160', '1177', '1178', '1181', '1187', '1188', '119', '1192', '1196', '1198', '1200', '121', '1226', '123', '1239', '1249', '1253', '1258', '126', '1277', '1279', '1283', '1285', '1302', '1303', '131', '1314', '1318', '1323', '1325', '133', '1332', '1334', '1336', '1346', '1347', '1348', '1353', '1366', '1367', '1373', '1377', '1408', '1413', '1420', '1421', '1422', '1425', '1427', '1428', '1430', '1433', '1434', '1435', '1438', '1440', '1441', '1445', '1450', '1452', '150', '1503', '1512', '1514', '1519', '1520', '1525', '1528', '153', '1530', '1533', '1536', '1539', '1546', '1547', '155', '1601', '1605', '1606', '1610', '1620', '1634', '1640', '1641', '165', '1655', '1659', '166', '1663', '1664', '1667', '1668', '167', '1671', '1672', '1679', '1680', '1685

In [114]:
station_over10_one=set(station_over10_week_one).intersection(set(station_over10_hol_one))
print(station_over10_one)
print('크기 : ',len(station_over10_one))

{'1822', '227', '902', '2351', '119', '1969', '1659', '1291', '606', '1726', '1730', '928', '1438', '1929', '1723', '1052', '1016', '1715', '205', '1413', '1661', '1428', '581', '1826', '1056', '808', '2356', '1039', '1855', '1952', '2344', '1054', '544', '1981', '2389', '1058', '1450', '2309', '1977', '2288', '1620', '1951', '2203', '1858', '923', '1953', '1053', '209', '2229', '1271', '1820', '966', '1511', '524', '1057', '1677', '1654', '221', '803', '2376', '2103', '1825', '1857', '1987', '959', '714', '2343', '1717', '2207', '1835', '2308'}
크기 :  71


In [115]:
station_over10_two=set(station_over10_week_two).intersection(set(station_over10_hol_two))
print(station_over10_two)
print('크기 : ',len(station_over10_two))

{'1722', '902', '606', '1664', '914', '1438', '1925', '2411', '1937', '3101', '1413', '1846', '2215', '1049', '652', '2368', '432', '1824', '1058', '1831', '1992', '938', '1366', '1814', '1830', '1850', '1858', '1640', '1641', '439', '1833', '314', '123', '1827', '1991', '1663', '175', '604', '1836', '2107', '119', '712', '2509', '650', '1726', '1181', '1420', '569', '1734', '1348', '1007', '2395', '1332', '133', '2339', '1188', '1691', '3528', '2265', '2501', '941', '1346', '2388', '524', '2311', '1825', '1736', '1987', '2012', '1434', '1373', '1421', '2372', '1435', '523', '915', '2024', '1325', '928', '1819', '1805', '1528', '2065', '1826', '1952', '2603', '166', '201', '2056', '1450', '1741', '1441', '2269', '1738', '1353', '2177', '564', '2287', '2403', '1347', '1367', '1452', '706', '1659', '1334', '1742', '1847', '1715', '2214', '937', '1445', '534', '2286', '182', '2307', '2309', '967', '1951', '1668', '1988', '2341', '947', '1512', '1006', '1038', '1057', '1967', '1928', '1318

### 분기별 대여소 이용권 비율 변화 확인

In [117]:
def make_quarter(df):
    first=[1,2,3]
    second=[4,5,6]
    third=[7,8,9]
    fourth=[10,11,12]
    
    df_first=df[df['month'].isin(first)].reset_index(drop=True)
    df_second=df[df['month'].isin(second)].reset_index(drop=True)
    df_third=df[df['month'].isin(third)].reset_index(drop=True)
    df_fourth=df[df['month'].isin(fourth)].reset_index(drop=True)
    
    return df_first, df_second, df_third, df_fourth

In [118]:
df2019_week1, df2019_week2, df2019_week3, df2019_week4=make_quarter(df2019_week)
df2020_week1, df2020_week2, df2020_week3, df2020_week4=make_quarter(df2020_week)

df2019_hol1, df2019_hol2, df2019_hol3, df2019_hol4=make_quarter(df2019_hol)
df2020_hol1, df2020_hol2, df2020_hol3, df2020_hol4=make_quarter(df2020_hol)

In [123]:
df2019_week_station_one=ratio_by_station(df2019_week1)
df2019_week_station_one.rename(columns={'Regular':'19_week_one'}, inplace=True)
df2019_week_station_two=ratio_by_station(df2019_week2)
df2019_week_station_two.rename(columns={'Regular':'19_week_two'}, inplace=True)
df2019_week_station_three=ratio_by_station(df2019_week3)
df2019_week_station_three.rename(columns={'Regular':'19_week_three'}, inplace=True)
df2019_week_station_four=ratio_by_station(df2019_week4)
df2019_week_station_four.rename(columns={'Regular':'19_week_four'}, inplace=True)

df2020_week_station_one=ratio_by_station(df2020_week1)
df2020_week_station_one.rename(columns={'Regular':'20_week_one'}, inplace=True)
df2020_week_station_two=ratio_by_station(df2020_week2)
df2020_week_station_two.rename(columns={'Regular':'20_week_two'}, inplace=True)
df2020_week_station_three=ratio_by_station(df2020_week3)
df2020_week_station_three.rename(columns={'Regular':'20_week_three'}, inplace=True)
df2020_week_station_four=ratio_by_station(df2020_week4)
df2020_week_station_four.rename(columns={'Regular':'20_week_four'}, inplace=True)

df2019_hol_station_one=ratio_by_station(df2019_hol1)
df2019_hol_station_one.rename(columns={'Regular':'19_hol_one'}, inplace=True)
df2019_hol_station_two=ratio_by_station(df2019_hol2)
df2019_hol_station_two.rename(columns={'Regular':'19_hol_two'}, inplace=True)
df2019_hol_station_three=ratio_by_station(df2019_hol3)
df2019_hol_station_three.rename(columns={'Regular':'19_hol_three'}, inplace=True)
df2019_hol_station_four=ratio_by_station(df2019_hol4)
df2019_hol_station_four.rename(columns={'Regular':'19_hol_four'}, inplace=True)

df2020_hol_station_one=ratio_by_station(df2020_hol1)
df2020_hol_station_one.rename(columns={'Regular':'20_hol_one'}, inplace=True)
df2020_hol_station_two=ratio_by_station(df2020_hol2)
df2020_hol_station_two.rename(columns={'Regular':'20_hol_two'}, inplace=True)
df2020_hol_station_three=ratio_by_station(df2020_hol3)
df2020_hol_station_three.rename(columns={'Regular':'20_hol_three'}, inplace=True)
df2020_hol_station_four=ratio_by_station(df2020_hol4)
df2020_hol_station_four.rename(columns={'Regular':'20_hol_four'}, inplace=True)


df_station=pd.merge(df2019_week_station_one, df2019_week_station_two, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2019_week_station_three, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2019_week_station_four, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_week_station_one, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_week_station_two, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_week_station_three, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_week_station_four, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2019_hol_station_one, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2019_hol_station_two, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2019_hol_station_three, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2019_hol_station_four, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_hol_station_one, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_hol_station_two, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_hol_station_three, left_on='station_id',right_on='station_id',how='outer')
df_station=pd.merge(df_station, df2020_hol_station_four, left_on='station_id',right_on='station_id',how='outer')

In [124]:
df_station.head()

,station_id,19_week_one,19_week_two,19_week_three,19_week_four,20_week_one,20_week_two,20_week_three,20_week_four,19_hol_one,19_hol_two,19_hol_three,19_hol_four,20_hol_one,20_hol_two,20_hol_three,20_hol_four
0,1001,0.877034,0.789884,0.830481,0.854372,0.771191,0.699906,0.748335,0.813953,0.755511,0.627308,0.704364,0.704380,0.667752,0.529262,0.639004,0.694611
1,1002,0.861982,0.758749,0.828142,0.847139,0.761905,0.656315,0.726898,0.820513,0.648438,0.570738,0.679766,0.672595,0.624309,0.476077,0.553086,0.591133
2,1003,0.875300,0.840941,0.829137,0.861865,0.828025,0.678705,0.743697,0.834286,0.786744,0.708383,0.742794,0.755043,0.748744,0.526807,0.612766,0.722892
3,1004,0.832051,0.797438,0.832990,0.857214,0.747695,0.694482,0.729097,0.845070,0.723164,0.684943,0.697086,0.706680,0.572727,0.546482,0.620283,0.555556
4,1006,0.919383,0.862541,0.886264,0.900209,0.812689,0.796710,0.784314,0.817204,0.816054,0.715649,0.727520,0.810662,0.657407,0.636580,0.649682,0.523810


In [136]:
df_station=df_station[df_station['station_id'].isin(id_info)].reset_index(drop=True)
df_station.head()

,station_id,19_week_one,19_week_two,19_week_three,19_week_four,20_week_one,20_week_two,20_week_three,20_week_four,19_hol_one,...,hol_three_diff,hol_four_diff,week one diff sign,week two diff sign,week three diff sign,week four diff sign,hol one diff sign,hol two diff sign,hol three diff sign,hol four diff sign
0,1001,0.877034,0.789884,0.830481,0.854372,0.771191,0.699906,0.748335,0.813953,0.755511,...,0.065360,0.009769,-,-,-,-,-,-,-,-
1,1002,0.861982,0.758749,0.828142,0.847139,0.761905,0.656315,0.726898,0.820513,0.648438,...,0.126680,0.081462,-,-,-,-,-,-,-,-
2,1003,0.875300,0.840941,0.829137,0.861865,0.828025,0.678705,0.743697,0.834286,0.786744,...,0.130028,0.032152,-,-,-,-,-,-,-,-
3,1004,0.832051,0.797438,0.832990,0.857214,0.747695,0.694482,0.729097,0.845070,0.723164,...,0.076803,0.151124,-,-,-,-,-,-,-,-
4,1006,0.919383,0.862541,0.886264,0.900209,0.812689,0.796710,0.784314,0.817204,0.816054,...,0.077839,0.286852,-,-,-,-,-,-,-,-


In [137]:
df_station.shape

(1527, 33)

In [138]:
df_station['week_one_diff']=abs(df_station['20_week_one']-df_station['19_week_one'])
df_station['week_two_diff']=abs(df_station['20_week_two']-df_station['19_week_two'])
df_station['week_three_diff']=abs(df_station['20_week_three']-df_station['19_week_three'])
df_station['week_four_diff']=abs(df_station['20_week_four']-df_station['19_week_four'])
df_station['hol_one_diff']=abs(df_station['20_hol_one']-df_station['19_hol_one'])
df_station['hol_two_diff']=abs(df_station['20_hol_two']-df_station['19_hol_two'])
df_station['hol_three_diff']=abs(df_station['20_hol_three']-df_station['19_hol_three'])
df_station['hol_four_diff']=abs(df_station['20_hol_four']-df_station['19_hol_four'])

df_station.head()

,station_id,19_week_one,19_week_two,19_week_three,19_week_four,20_week_one,20_week_two,20_week_three,20_week_four,19_hol_one,...,hol_three_diff,hol_four_diff,week one diff sign,week two diff sign,week three diff sign,week four diff sign,hol one diff sign,hol two diff sign,hol three diff sign,hol four diff sign
0,1001,0.877034,0.789884,0.830481,0.854372,0.771191,0.699906,0.748335,0.813953,0.755511,...,0.065360,0.009769,-,-,-,-,-,-,-,-
1,1002,0.861982,0.758749,0.828142,0.847139,0.761905,0.656315,0.726898,0.820513,0.648438,...,0.126680,0.081462,-,-,-,-,-,-,-,-
2,1003,0.875300,0.840941,0.829137,0.861865,0.828025,0.678705,0.743697,0.834286,0.786744,...,0.130028,0.032152,-,-,-,-,-,-,-,-
3,1004,0.832051,0.797438,0.832990,0.857214,0.747695,0.694482,0.729097,0.845070,0.723164,...,0.076803,0.151124,-,-,-,-,-,-,-,-
4,1006,0.919383,0.862541,0.886264,0.900209,0.812689,0.796710,0.784314,0.817204,0.816054,...,0.077839,0.286852,-,-,-,-,-,-,-,-


In [139]:
df_station['week one diff sign']=df_station.apply(lambda x:make_sign(x['19_week_one'], x['20_week_one']), axis=1)
df_station['week two diff sign']=df_station.apply(lambda x:make_sign(x['19_week_two'], x['20_week_two']), axis=1)
df_station['week three diff sign']=df_station.apply(lambda x:make_sign(x['19_week_three'], x['20_week_three']), axis=1)
df_station['week four diff sign']=df_station.apply(lambda x:make_sign(x['19_week_four'], x['20_week_four']), axis=1)
df_station['hol one diff sign']=df_station.apply(lambda x:make_sign(x['19_hol_one'], x['20_hol_one']), axis=1)
df_station['hol two diff sign']=df_station.apply(lambda x:make_sign(x['19_hol_two'], x['20_hol_two']), axis=1)
df_station['hol three diff sign']=df_station.apply(lambda x:make_sign(x['19_hol_three'], x['20_hol_three']), axis=1)
df_station['hol four diff sign']=df_station.apply(lambda x:make_sign(x['19_hol_four'], x['20_hol_four']), axis=1)

df_station.head()

,station_id,19_week_one,19_week_two,19_week_three,19_week_four,20_week_one,20_week_two,20_week_three,20_week_four,19_hol_one,...,hol_three_diff,hol_four_diff,week one diff sign,week two diff sign,week three diff sign,week four diff sign,hol one diff sign,hol two diff sign,hol three diff sign,hol four diff sign
0,1001,0.877034,0.789884,0.830481,0.854372,0.771191,0.699906,0.748335,0.813953,0.755511,...,0.065360,0.009769,-,-,-,-,-,-,-,-
1,1002,0.861982,0.758749,0.828142,0.847139,0.761905,0.656315,0.726898,0.820513,0.648438,...,0.126680,0.081462,-,-,-,-,-,-,-,-
2,1003,0.875300,0.840941,0.829137,0.861865,0.828025,0.678705,0.743697,0.834286,0.786744,...,0.130028,0.032152,-,-,-,-,-,-,-,-
3,1004,0.832051,0.797438,0.832990,0.857214,0.747695,0.694482,0.729097,0.845070,0.723164,...,0.076803,0.151124,-,-,-,-,-,-,-,-
4,1006,0.919383,0.862541,0.886264,0.900209,0.812689,0.796710,0.784314,0.817204,0.816054,...,0.077839,0.286852,-,-,-,-,-,-,-,-


In [140]:
# 정기권의 이용 비율이 증가한 대여소 (주중 1분기)

station_week_one_plus=df_station[df_station['week one diff sign']=='+']['station_id'].tolist()
print(station_week_one_plus)
print('크기 : ',len(station_week_one_plus))

['1007', '1025', '1032', '1060', '1168', '117', '1170', '1177', '1180', '1181', '1182', '1184', '1186', '1187', '1191', '1192', '1195', '1196', '1198', '1200', '1251', '1263', '1304', '1325', '1349', '135', '1366', '1369', '1370', '1402', '1406', '1411', '1424', '1441', '1501', '1527', '1536', '1543', '1662', '1678', '1736', '1806', '1836', '1849', '1853', '1910', '1916', '1920', '1930', '1946', '1971', '1993', '200', '2036', '2041', '2061', '2062', '2137', '2141', '2164', '2225', '2235', '2272', '2275', '2289', '2292', '2293', '2319', '2352', '2357', '2381', '2385', '2390', '2392', '2503', '2511', '289', '312', '324', '337', '344', '3501', '3539', '354', '359', '379', '401', '420', '428', '434', '435', '449', '450', '526', '562', '575', '591', '604', '626', '639', '750', '754', '817', '846', '924', '932', '942', '949', '961', '963', '968']
크기 :  111


In [141]:
# 정기권의 이용 비율이 증가한 대여소 (주중 2분기)

station_week_two_plus=df_station[df_station['week two diff sign']=='+']['station_id'].tolist()
print(station_week_two_plus)
print('크기 : ',len(station_week_two_plus))

['1018', '1025', '1183', '1186', '1296', '1297', '1304', '135', '1370', '1546', '1646', '1652', '1679', '1731', '180', '181', '198', '2041', '2104', '2141', '2225', '2245', '2270', '2275', '2289', '2299', '2329', '2364', '2371', '2385', '2392', '2395', '2407', '2510', '2601', '2607', '2619', '337', '3529', '391', '424', '428', '455', '575', '656', '748', '802', '831', '969', '1376', '1377', '462', '972']
크기 :  53


In [142]:
# 정기권의 이용 비율이 증가한 대여소 (주중 3분기)

station_week_three_plus=df_station[df_station['week three diff sign']=='+']['station_id'].tolist()
print(station_week_three_plus)
print('크기 : ',len(station_week_three_plus))

['1018', '1025', '1027', '1035', '1101', '1183', '1215', '1218', '1239', '1241', '1267', '1268', '1296', '1297', '1304', '1352', '1370', '1510', '1511', '1513', '1547', '1652', '1680', '1728', '1730', '1936', '1969', '1971', '2008', '2061', '2064', '2143', '218', '2207', '2226', '2275', '2283', '2298', '2333', '2343', '2391', '2392', '2609', '2618', '320', '3507', '3521', '430', '455', '575', '616', '802', '810', '968', '1376']
크기 :  55


In [148]:
# 정기권의 이용 비율이 증가한 대여소 (주중 4분기)

station_week_four_plus=df_station[df_station['week four diff sign']=='+']['station_id'].tolist()
print(station_week_four_plus)
print('크기 : ',len(station_week_four_plus))

['101', '1010', '1012', '1014', '1024', '1025', '1026', '1029', '1030', '1034', '1039', '104', '1041', '1048', '105', '1054', '1056', '1109', '1110', '1126', '1127', '1128', '1130', '1137', '1139', '1146', '1149', '1151', '1152', '1157', '1159', '1180', '1193', '1199', '121', '122', '1228', '1230', '1235', '1239', '1240', '1241', '1242', '1256', '1258', '1263', '1266', '1267', '1268', '1279', '1282', '1283', '1290', '1303', '1316', '134', '1352', '1359', '1360', '138', '140', '1416', '142', '1422', '1429', '143', '1430', '1443', '1459', '147', '1509', '1510', '1511', '1513', '1518', '1532', '1619', '163', '1650', '1671', '1678', '1714', '173', '180', '1806', '1820', '1855', '1916', '1933', '1936', '1943', '196', '1965', '1969', '1973', '199', '1999', '203', '2058', '2064', '2067', '210', '2104', '2110', '2119', '2122', '2129', '2139', '2141', '2145', '2148', '2167', '2184', '2201', '2214', '2222', '2226', '223', '2230', '2231', '2235', '2239', '2242', '2244', '2250', '2257', '2280', '2

In [149]:
# 정기권의 이용 비율이 증가한 대여소 (주말 1분기)

station_hol_one_plus=df_station[df_station['hol one diff sign']=='+']['station_id'].tolist()
print(station_hol_one_plus)
print('크기 : ',len(station_hol_one_plus))

['1007', '1042', '1060', '1061', '1105', '1108', '1110', '1112', '1113', '1114', '1117', '1135', '1146', '1151', '1157', '1165', '1166', '1168', '1170', '1173', '1180', '1182', '1184', '1188', '1190', '1191', '1192', '1193', '1196', '1198', '1200', '1221', '1224', '127', '1277', '1279', '1282', '1302', '1304', '1305', '1321', '1326', '1333', '1340', '1348', '1360', '1366', '1368', '137', '1402', '1435', '1441', '1445', '1447', '1456', '1459', '147', '1501', '1510', '1515', '1525', '1526', '1527', '153', '1534', '1536', '1542', '1543', '1546', '1548', '1603', '163', '1650', '1660', '1662', '1667', '1670', '1690', '1719', '1725', '1741', '181', '1826', '1830', '1836', '1839', '1845', '1847', '1848', '1854', '1855', '1859', '188', '1910', '1912', '1946', '1956', '1966', '1968', '1971', '1975', '1983', '1992', '1993', '2012', '2025', '2026', '2037', '2058', '2061', '2068', '2109', '2115', '2127', '2129', '2130', '2141', '2145', '2156', '2172', '2180', '2231', '2239', '2244', '2246', '2251'

In [150]:
# 정기권의 이용 비율이 증가한 대여소 (주말 2분기)

station_hol_two_plus=df_station[df_station['hol two diff sign']=='+']['station_id'].tolist()
print(station_hol_two_plus)
print('크기 : ',len(station_hol_two_plus))

['1108', '1135', '1186', '1203', '1204', '1211', '1220', '1227', '1229', '1231', '1242', '1256', '1260', '1283', '1296', '1297', '1304', '1316', '1338', '1342', '136', '137', '1370', '1372', '1412', '1435', '1510', '1652', '1692', '1707', '1808', '181', '196', '1967', '1976', '199', '1993', '2036', '2058', '2063', '2068', '2104', '2127', '2141', '2210', '2221', '2233', '2244', '2250', '2258', '2266', '2277', '2297', '2302', '2328', '2333', '2335', '2349', '2355', '2369', '2406', '2409', '2502', '260', '2601', '2606', '2610', '2612', '2615', '2619', '307', '310', '337', '3501', '3506', '3507', '3529', '391', '402', '417', '424', '452', '455', '535', '575', '636', '741', '742', '750', '759', '802', '812', '846', '900', '963', '1376', '972']
크기 :  97


In [151]:
# 정기권의 이용 비율이 증가한 대여소 (주말 3분기)

station_hol_three_plus=df_station[df_station['hol three diff sign']=='+']['station_id'].tolist()
print(station_hol_three_plus)
print('크기 : ',len(station_hol_three_plus))

['1013', '1027', '1036', '1053', '1108', '1119', '1151', '1183', '1194', '1205', '1206', '1215', '1220', '1229', '1231', '1233', '1235', '1242', '1259', '1262', '1263', '1268', '1293', '1304', '1306', '1316', '1357', '136', '1362', '1364', '1365', '1370', '1375', '1444', '1510', '1534', '159', '160', '1628', '1630', '1652', '1670', '171', '179', '1809', '196', '1979', '1983', '1993', '2016', '2041', '2057', '2062', '2064', '2111', '2127', '2129', '2130', '2143', '2203', '2211', '2225', '2233', '2239', '2242', '2259', '2266', '2272', '2280', '2284', '2288', '2290', '2297', '2304', '2306', '2330', '2343', '235', '2361', '2362', '2381', '2385', '2391', '2406', '2409', '242', '2502', '2510', '2607', '2609', '2612', '2618', '285', '305', '322', '332', '337', '339', '3527', '3529', '354', '3542', '368', '375', '378', '389', '393', '394', '397', '398', '410', '412', '449', '454', '455', '456', '501', '514', '522', '533', '540', '543', '555', '560', '564', '574', '575', '608', '610', '638', '6

In [152]:
# 정기권의 이용 비율이 증가한 대여소 (주말 4분기)

station_hol_four_plus=df_station[df_station['hol four diff sign']=='+']['station_id'].tolist()
print(station_hol_four_plus)
print('크기 : ',len(station_hol_four_plus))

['101', '1012', '1015', '1018', '1024', '1025', '1029', '1034', '105', '1054', '1056', '1057', '1101', '1102', '1113', '115', '1151', '1165', '1166', '1183', '1190', '1193', '1203', '1210', '1214', '122', '1221', '1228', '1231', '1233', '1241', '1250', '1257', '1260', '1266', '1267', '1268', '127', '1275', '1289', '129', '1300', '1304', '1316', '1322', '134', '1342', '1344', '1353', '139', '140', '1407', '141', '1414', '1418', '1424', '1443', '1444', '1448', '1454', '1457', '1510', '1513', '1531', '1540', '1545', '155', '158', '160', '164', '1660', '1661', '1675', '1677', '169', '170', '1705', '171', '173', '1731', '1732', '179', '1816', '1820', '1828', '1836', '1840', '1842', '1849', '1856', '1859', '1907', '1910', '195', '198', '199', '2041', '2050', '2061', '2069', '2071', '210', '2111', '2127', '2135', '2139', '214', '2144', '2148', '2164', '2165', '2174', '2178', '218', '2225', '2226', '2241', '2243', '2257', '2259', '2262', '2284', '2292', '2299', '2305', '231', '2314', '2323', '

In [154]:
station_one_plus=set(station_week_one_plus).intersection(set(station_hol_one_plus))
print(station_one_plus)
print('크기 :',len(station_one_plus))

{'1192', '604', '1836', '2390', '324', '1662', '1304', '2319', '1168', '2503', '1402', '575', '1198', '1946', '1191', '1971', '942', '963', '2293', '1993', '1527', '354', '1007', '1180', '2061', '932', '3539', '2289', '2141', '1060', '1441', '1366', '2272', '1170', '846', '1184', '1196', '289', '1182', '359', '435', '1501', '2392', '1200', '1543', '1536', '344', '1910'}
크기 : 48


In [156]:
station_two_plus=set(station_week_two_plus).intersection(set(station_hol_two_plus))
print(station_two_plus)
print('크기 :',len(station_two_plus))

{'1297', '1304', '2619', '575', '1652', '1186', '3529', '972', '455', '337', '2104', '391', '2141', '424', '802', '1376', '1370', '1296', '2601', '181'}
크기 : 20


In [157]:
station_three_plus=set(station_week_three_plus).intersection(set(station_hol_three_plus))
print(station_three_plus)
print('크기 :',len(station_three_plus))

{'1268', '455', '1027', '1183', '1510', '2143', '2618', '1215', '2064', '802', '2343', '1370', '1304', '2391', '2609', '575', '1652'}
크기 : 17


In [158]:
station_four_plus=set(station_week_four_plus).intersection(set(station_hol_four_plus))
print(station_four_plus)
print('크기 :',len(station_four_plus))

{'1268', '1012', '428', '1034', '322', '173', '1241', '338', '267', '2139', '242', '2391', '140', '105', '1513', '1510', '270', '210', '2257', '1316', '2350', '1056', '1228', '1054', '1266', '101', '1151', '449', '122', '1025', '455', '508', '387', '710', '447', '3531', '820', '1029', '952', '134', '1267', '244', '2323', '2502', '1193', '846', '394', '2226', '199', '331', '320', '422', '543', '1820', '2148', '430', '634', '1443', '1024', '429', '806', '380', '2343', '2284', '349', '818', '2364', '346'}
크기 : 68


#### 10% 이상의 이용권 비율차이가 있는 대여소 목록

In [159]:
station_over10_week_one=df_station[df_station['week_one_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_week_one)
print('크기 : ',len(station_over10_week_one))

['1001', '1002', '1006', '1007', '1016', '103', '1034', '1035', '1039', '104', '1052', '1054', '1056', '1057', '1058', '106', '1062', '1106', '111', '1114', '112', '1122', '1127', '1129', '1148', '1161', '1169', '117', '1175', '118', '1183', '119', '1231', '1250', '1268', '1271', '1274', '1287', '1300', '131', '1315', '1323', '1332', '1337', '1357', '1358', '1359', '136', '1373', '1408', '1413', '1420', '1424', '1437', '1438', '144', '1450', '1451', '1452', '1453', '1454', '1455', '1512', '152', '1534', '1536', '1547', '1601', '1602', '1608', '1640', '1641', '1643', '1659', '1661', '1663', '1664', '1675', '1677', '1703', '1705', '1714', '1715', '1717', '1720', '1722', '1726', '1729', '1735', '1742', '175', '181', '1816', '182', '1821', '1824', '1825', '1827', '1833', '1834', '1845', '185', '1852', '1858', '186', '1901', '1925', '1928', '1933', '1937', '1951', '1952', '1953', '1959', '1969', '1979', '1987', '199', '1991', '2000', '2002', '201', '2015', '2024', '2027', '2038', '2040', '2

In [160]:
station_over10_week_two=df_station[df_station['week_two_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_week_two)
print('크기 : ',len(station_over10_week_two))

['1002', '1003', '1004', '1007', '1016', '1018', '1021', '1035', '1037', '1038', '1039', '1049', '1052', '1053', '1054', '1056', '1057', '1058', '1114', '112', '1121', '1122', '1130', '1131', '1132', '1140', '1148', '1157', '1165', '117', '1172', '1178', '1181', '119', '1200', '1207', '121', '1213', '1228', '123', '1271', '1275', '1291', '131', '1314', '1315', '1332', '1347', '1349', '1359', '1367', '1373', '1406', '1408', '1411', '1413', '1414', '1418', '142', '1421', '1428', '1429', '1430', '1433', '1434', '1435', '1437', '1438', '1444', '1447', '1450', '1452', '1453', '148', '1510', '1511', '1514', '1516', '152', '1520', '1525', '1528', '1533', '1545', '1547', '1548', '1601', '1602', '1603', '1608', '1620', '1628', '1634', '1637', '1640', '1641', '1651', '1656', '1657', '1659', '1661', '1664', '1666', '1667', '1668', '1671', '1672', '1677', '1681', '1683', '1685', '1691', '1705', '1708', '1715', '1716', '1717', '1719', '1720', '1722', '1723', '1726', '1730', '1735', '1738', '1804', 

In [161]:
station_over10_week_three=df_station[df_station['week_three_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_week_three)
print('크기 : ',len(station_over10_week_three))

['1002', '1004', '1006', '1018', '1035', '1037', '1038', '1049', '1052', '1053', '1056', '1057', '1058', '106', '1060', '1109', '1139', '117', '1170', '1178', '118', '1188', '119', '1229', '123', '1271', '1290', '1318', '1325', '133', '1334', '1347', '1348', '1349', '1353', '1359', '1366', '1367', '1373', '1413', '1418', '1420', '1421', '1427', '1429', '1435', '1436', '1437', '1438', '1450', '1452', '1512', '152', '1528', '1534', '1537', '1541', '159', '162', '1640', '1641', '1659', '166', '1661', '1663', '1668', '167', '1671', '1677', '1691', '170', '1702', '1708', '1715', '1720', '1722', '1726', '1734', '1736', '1738', '1741', '1742', '175', '178', '1805', '1814', '1816', '1819', '182', '1820', '1824', '1825', '1826', '1827', '1830', '1833', '1835', '1836', '1841', '1846', '1847', '1850', '1853', '1855', '1857', '1858', '1859', '186', '1925', '1928', '1937', '194', '1951', '1952', '1965', '1973', '1981', '1985', '1987', '1988', '1991', '1996', '1999', '201', '2012', '2024', '2038', '

In [162]:
station_over10_week_four=df_station[df_station['week_four_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_week_four)
print('크기 : ',len(station_over10_week_four))

['1007', '1015', '1031', '1032', '1038', '1042', '1057', '1058', '111', '1121', '115', '1161', '117', '1181', '1188', '119', '1285', '1287', '1291', '1314', '1315', '1318', '133', '1332', '1336', '1337', '1346', '1347', '1348', '1357', '1367', '139', '1401', '1406', '1407', '1412', '1413', '1418', '1420', '1421', '1422', '1425', '1434', '1435', '1438', '1445', '1453', '1458', '1504', '1513', '1516', '1522', '1528', '1536', '1539', '1548', '1603', '1611', '1634', '1640', '1641', '1651', '1654', '1663', '1664', '1665', '1668', '1692', '1706', '1715', '1722', '1725', '1726', '1730', '1734', '1736', '1742', '1814', '1819', '182', '1823', '1824', '1826', '1827', '1833', '1839', '1846', '1850', '1857', '1858', '1930', '1937', '1952', '1964', '1967', '1974', '1983', '1992', '1998', '1999', '2008', '201', '2025', '2034', '2056', '2116', '2135', '2143', '2159', '216', '2203', '2205', '2207', '2215', '2223', '2227', '2241', '2248', '2258', '2263', '2265', '2269', '2276', '2287', '2289', '2304', 

In [164]:
station_over10_hol_one=df_station[df_station['hol_one_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_hol_one)
print('크기 : ',len(station_over10_hol_one))

['1004', '1006', '1008', '1009', '1010', '1015', '1016', '1017', '1018', '1020', '1023', '1026', '1029', '1031', '1033', '1034', '1035', '1038', '1039', '1041', '1054', '1056', '1057', '1058', '106', '1060', '109', '1101', '111', '1120', '1129', '1137', '1152', '1159', '1161', '1171', '1175', '118', '1183', '119', '1207', '1209', '121', '1215', '1226', '1227', '1228', '1233', '1235', '1239', '1242', '1243', '1246', '1249', '125', '1258', '1263', '1271', '1285', '1286', '1287', '1291', '1300', '1334', '1337', '1342', '1347', '1348', '1349', '1355', '1358', '136', '1362', '1366', '1413', '1416', '1420', '1421', '1426', '1428', '1429', '1430', '1431', '1433', '1438', '1443', '1444', '1445', '1449', '145', '1451', '148', '1511', '1513', '152', '1528', '1533', '1545', '1547', '155', '1608', '164', '1640', '1643', '1646', '165', '1659', '1661', '1664', '1665', '167', '1671', '1672', '1677', '1678', '1679', '1680', '1685', '1702', '1714', '1715', '1717', '1722', '1726', '1727', '1728', '1729'

In [165]:
station_over10_hol_two=df_station[df_station['hol_two_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_hol_two)
print('크기 : ',len(station_over10_hol_two))

['1003', '1004', '1008', '101', '1015', '1016', '1017', '1023', '1031', '1032', '1033', '1038', '1039', '1041', '1042', '1047', '1049', '1052', '1053', '1054', '1056', '1057', '1058', '1062', '109', '110', '1102', '1105', '1113', '1120', '1121', '1126', '1146', '1148', '1150', '1151', '1155', '1160', '1161', '1163', '1165', '1172', '1174', '1175', '1178', '1181', '1183', '1186', '1188', '119', '1200', '1207', '121', '1228', '1235', '1246', '1250', '1263', '1266', '1271', '1275', '1280', '1285', '1291', '131', '1314', '1318', '1319', '1323', '1325', '1327', '133', '1332', '134', '1346', '1347', '1349', '1353', '1357', '1360', '1366', '1367', '1370', '1401', '1402', '1404', '1405', '1406', '1408', '1410', '1411', '1413', '1416', '1418', '1420', '1421', '1422', '1424', '1428', '1429', '1434', '1437', '1438', '1441', '1442', '1443', '1444', '1445', '1447', '1450', '1452', '1453', '1455', '1459', '148', '1503', '1506', '1509', '1511', '1512', '1513', '1514', '1516', '1519', '1520', '1525', 

In [166]:
station_over10_hol_three=df_station[df_station['hol_three_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_hol_three)
print('크기 : ',len(station_over10_hol_three))

['1002', '1003', '1007', '1018', '1031', '1032', '1033', '1037', '1038', '1042', '1049', '1052', '1057', '1058', '106', '1062', '108', '109', '1105', '1109', '111', '1115', '1124', '1137', '1148', '115', '1150', '1152', '1155', '1160', '1165', '1169', '1178', '1181', '1187', '119', '1190', '1198', '1200', '1207', '121', '1226', '1239', '124', '1249', '1250', '1253', '126', '1271', '1283', '1285', '1302', '1306', '131', '1314', '1318', '1323', '1325', '1326', '133', '1334', '1336', '1346', '1347', '1348', '1353', '1360', '1365', '1367', '139', '1408', '141', '1413', '1421', '1422', '1424', '1427', '1428', '1429', '1430', '1433', '1435', '1438', '1440', '1441', '1452', '150', '1503', '1506', '1512', '1514', '1519', '1520', '1525', '1528', '153', '1530', '1533', '1536', '1537', '1539', '1546', '1547', '1548', '155', '157', '1601', '1606', '1610', '1620', '1634', '1640', '165', '1655', '1656', '1659', '1661', '1663', '1664', '1667', '167', '1671', '1677', '1679', '1680', '169', '1691', '17

In [167]:
station_over10_hol_four=df_station[df_station['hol_four_diff'] >= 0.1]['station_id'].tolist()
print(station_over10_hol_four)
print('크기 : ',len(station_over10_hol_four))

['1004', '1006', '1007', '1010', '1011', '1016', '1018', '1023', '1031', '1032', '1033', '1037', '1038', '1039', '1042', '1047', '1048', '1049', '1051', '1054', '1058', '1061', '1062', '1101', '1105', '111', '1115', '1117', '1120', '1122', '1124', '1125', '1132', '1141', '1145', '1148', '1152', '1153', '1155', '1157', '1158', '1159', '1160', '1162', '1167', '1173', '1174', '1177', '118', '1181', '1187', '1188', '119', '1192', '1194', '1195', '1196', '1201', '1205', '121', '1222', '1226', '1227', '123', '1233', '1240', '1241', '1245', '1248', '1249', '1253', '1258', '126', '1262', '1263', '1265', '127', '1274', '1277', '1279', '1285', '1286', '1287', '129', '1291', '1299', '1303', '131', '1314', '1315', '1321', '1323', '1325', '133', '1332', '1333', '1336', '1339', '1340', '1342', '1346', '1348', '1355', '1357', '1361', '1364', '1366', '1369', '1373', '1401', '1402', '1404', '1408', '1410', '1413', '1420', '1421', '1422', '1425', '1426', '1428', '1430', '1431', '1434', '1435', '1436', '

In [168]:
df_station['hol_four_diff'].describe()

count    1508.000000
mean        0.094172
std         0.070151
min         0.000000
25%         0.041408
50%         0.080582
75%         0.129617
max         0.561905
Name: hol_four_diff, dtype: float64

In [169]:
station_over10_one=set(station_over10_week_one).intersection(set(station_over10_hol_one))
print(station_over10_one)
print('크기 :',len(station_over10_one))

{'1608', '1722', '902', '2201', '1034', '272', '1664', '1438', '278', '1016', '2411', '1937', '373', '1413', '1661', '1056', '2215', '652', '1039', '838', '2344', '1451', '1824', '1058', '2622', '938', '1858', '1640', '1953', '233', '209', '422', '749', '1337', '249', '832', '446', '221', '803', '2103', '711', '2222', '111', '227', '175', '1933', '119', '2387', '1161', '136', '1726', '2314', '830', '729', '205', '1420', '2038', '808', '248', '762', '260', '1183', '1729', '539', '2027', '524', '1825', '561', '1987', '2060', '2207', '2308', '1547', '2217', '915', '2613', '627', '2024', '954', '267', '928', '2609', '722', '825', '2356', '1952', '544', '502', '536', '1714', '201', '583', '271', '3539', '2141', '1175', '406', '1852', '565', '1677', '564', '152', '2000', '2287', '2262', '106', '1659', '1901', '1715', '2214', '588', '2015', '1035', '1054', '3525', '1287', '182', '2307', '185', '1129', '1951', '118', '2220', '1271', '2341', '1979', '1358', '1006', '1057', '1643', '2343', '2054

In [170]:
station_over10_two=set(station_over10_week_two).intersection(set(station_over10_hol_two))
print(station_over10_two)
print('크기 :',len(station_over10_two))

{'1822', '1722', '1520', '902', '912', '606', '2391', '1730', '1525', '1438', '1016', '707', '1413', '1846', '1661', '1056', '1049', '1039', '838', '2344', '2170', '585', '1058', '1831', '1514', '905', '1634', '1172', '1533', '1814', '1830', '148', '1429', '1053', '1640', '1656', '1858', '1953', '209', '1406', '221', '1437', '1833', '2103', '3106', '314', '429', '959', '252', '2059', '227', '604', '2107', '119', '1667', '1004', '1447', '1726', '1444', '1929', '1708', '1181', '205', '1428', '735', '235', '1855', '592', '2395', '1332', '1121', '1637', '1628', '2116', '2288', '1691', '3528', '2265', '2212', '524', '292', '2376', '1825', '1685', '1601', '206', '1434', '2112', '2308', '1421', '1178', '1418', '1547', '1516', '1859', '1973', '615', '1969', '915', '2024', '1931', '1941', '928', '1723', '1528', '827', '192', '2165', '581', '1826', '1228', '2356', '921', '1952', '2603', '544', '2389', '2056', '740', '1450', '237', '3539', '1996', '1977', '2102', '1411', '1738', '923', '1904', '1

In [171]:
station_over10_three=set(station_over10_week_three).intersection(set(station_over10_hol_three))
print(station_over10_three)
print('크기 :',len(station_over10_three))

{'1037', '1722', '2201', '1438', '2144', '167', '1925', '2411', '1002', '1937', '1413', '1846', '1661', '2215', '1049', '652', '2368', '432', '1824', '1058', '1814', '1850', '1858', '1429', '1640', '439', '1833', '711', '1109', '1827', '1991', '1663', '175', '381', '604', '1836', '1018', '119', '2107', '712', '2509', '1726', '569', '2038', '735', '1855', '1348', '1999', '2395', '1427', '133', '2289', '2339', '1691', '3528', '2265', '2501', '941', '2388', '2311', '1825', '1736', '2264', '1178', '1421', '1973', '1435', '523', '2024', '1325', '928', '1819', '1528', '827', '2065', '557', '1826', '1952', '2603', '201', '2056', '1996', '1741', '1738', '1353', '2177', '1671', '1677', '1857', '2159', '2287', '2403', '106', '1347', '420', '1367', '1452', '706', '1659', '1334', '1052', '1742', '1847', '2214', '937', '534', '2286', '1981', '1537', '3531', '182', '967', '186', '1951', '1988', '1271', '2341', '1820', '947', '1512', '1038', '1057', '1928', '1318', '2054', '1835', '643', '1965', '907

In [172]:
station_over10_four=set(station_over10_week_four).intersection(set(station_over10_hol_four))
print(station_over10_four)
print('크기 :',len(station_over10_four))

{'964', '1722', '1425', '1823', '2391', '1730', '1664', '914', '1438', '1513', '1937', '3101', '1413', '1846', '501', '1824', '1058', '216', '2143', '1401', '1634', '1992', '2304', '1850', '1858', '1640', '1539', '1641', '533', '1336', '438', '1422', '1833', '314', '1827', '111', '1032', '119', '2387', '650', '1181', '1420', '569', '1734', '1348', '1999', '449', '1007', '2395', '1332', '2324', '133', '1188', '3511', '2265', '1031', '1346', '2008', '1522', '1458', '1736', '1434', '2354', '1974', '1421', '2372', '2383', '1285', '2396', '1435', '523', '2321', '1819', '1528', '825', '1826', '656', '2356', '1952', '544', '1357', '201', '1725', '2056', '271', '2269', '556', '2205', '923', '1654', '1692', '1536', '2363', '2287', '564', '706', '2025', '1291', '1715', '1314', '1453', '1445', '2305', '2373', '1603', '534', '1042', '1839', '1287', '182', '2263', '1315', '2307', '2309', '759', '1668', '1983', '1038', '1967', '3105', '966', '1504', '907'}
크기 : 123
